# Práctica 2 - Data Mining, por Franco Bagnasco

Para esta tarea, veremos los datos de vacunas de 2017. El archivo con estos datos está en `/data/NISPUF17.csv`. Para conocer los nombres de las variables de los datos tienes disponible una guía de usuarios de datos en `/data/NIS-PUF17-DUG.pdf`.

## Pregunta 1

Escribe una función llamada `proportion_of_education` que devuelva la proporción de niños en el dataset que tuvieron una madre con los niveles de educación: igual a menor de secundaria (<12), secundaria (12), mayor que secundaria pero sin ser universitaria graduada (>12) y con título universitario.

Esta función debería devolver un diccionario en forma de (use los números correctos, no redondee los números):

```python
{"menor que  secundaria": 0.2,
 "secundaria": 0.4,
 "mayor a secundaria pero no universitaria": 0.2,
 "universitaria": 0.2}
```


In [75]:
import pandas as pd

def proportion_of_education():
    df = pd.read_csv("../data/NISPUF17.csv")

    # Filtramos el DataFrame para obtener solo la columna 'EDUC1' (nivel de educación de la madre)
    education_column = df['EDUC1'].dropna()

    # Creamos un diccionario para dividir los valores de EDUC1 en las 4 categorías
    education_mapping = {
        1: "menor que secundaria",
        2: "secundaria",
        3: "mayor a secundaria pero no universitaria",
        4: "universitaria"
    }

    # Usamos la función map para asignar los valores de EDUC1 a las categorías
    education_column = education_column.map(education_mapping)

    # Calculamos las proporciones para cada categoría de educación
    education_counts = education_column.value_counts(normalize=True).sort_index()

    return education_counts.to_dict()

# Llamar a la función y mostrar los resultados
result = proportion_of_education()
print(result)

{'mayor a secundaria pero no universitaria': 0.24588090637625154, 'menor que secundaria': 0.10202002459160373, 'secundaria': 0.172352011241876, 'universitaria': 0.47974705779026877}


## Pregunta 2

Exploremos la relación entre ser alimentado con leche materna (fed breast milk) cuando era niño y recibir una vacuna contra la influenza estacional (seasonal influenza) de un proveedor de atención médica.

Devuelva una tupla del número promedio de vacunas contra la influenza para aquellos niños que sabemos que recibieron leche materna cuando eran niños y aquellos que sabemos que no.

In [76]:
import pandas as pd

def average_influenza_doses():
    df = pd.read_csv('../data/NISPUF17.csv')

    # Filtrar las columnas relevantes y eliminar filas con valores nulos
    filtrado = df[['CBF_01', 'P_NUMFLU']].dropna()

    # Calcular el promedio de dosis de vacuna para el grupo que recibió leche materna
    promedio_leche_materna = filtrado[filtrado['CBF_01'] == 1]['P_NUMFLU'].mean()

    # Calcular el promedio de dosis de vacuna para el grupo que no recibió leche materna
    promedio_no_leche_materna = filtrado[filtrado['CBF_01'] == 2]['P_NUMFLU'].mean()

    # Interpretación de los resultados
    interpretacion = "En promedio, los niños que recibieron leche materna fueron vacunados {:.2f} veces contra la influenza estacional, mientras que los niños que no recibieron leche materna fueron vacunados {:.2f} veces.".format(promedio_leche_materna, promedio_no_leche_materna)
    
    # Devolver una tupla con los promedios calculados y la interpretación
    return (promedio_leche_materna, promedio_no_leche_materna, interpretacion)

# Llamar a la función y mostrar los resultados
result = average_influenza_doses()
print(result[:-1])
print(result[-1]) # Interpretación de la tupla

(1.8799187420058687, 1.5963945918878317)
En promedio, los niños que recibieron leche materna fueron vacunados 1.88 veces contra la influenza estacional, mientras que los niños que no recibieron leche materna fueron vacunados 1.60 veces.


# Pregunta 3

Sería interesante ver si hay evidencia de un vínculo entre la efectividad de la vacuna y el sexo del niño.
Calcule el ratio entre el número de niños que contrajeron varicela (chickenpox) pero que fueron vacunados contra ella (al menos una dosis) versus los que fueron vacunados y no contrajeron varicela.

Devuelve los resultados por sexo.

Esta función debería devolver un diccionario en forma de:

```python
{"male": 0.2, "female": 0.4}
```

Nota: Para ayudar en la verificación, el valor de chickenpox_by_sex()['female'] comienza con los dígitos 0.0077.

In [77]:
import pandas as pd

def chickenpox_by_sex():
    df = pd.read_csv('../data/NISPUF17.csv')

    # Contar el número de niños que contrajeron varicela después de estar vacunados (HAD_CPOX=1) y los que fueron vacunados pero no contrajeron varicela (HAD_CPOX=2)
    vaccinated_boys_with_chickenpox = len(df[(df['SEX'] == 1) & (df['P_NUMVRC'] >= 1) & (df['HAD_CPOX'] == 1)])
    vaccinated_boys_without_chickenpox = len(df[(df['SEX'] == 1) & (df['P_NUMVRC'] >= 1) & (df['HAD_CPOX'] == 2)])

    # Contar el número de niñas que contrajeron varicela después de estar vacunadas (HAD_CPOX=1) y las que fueron vacunadas pero no contrajeron varicela (HAD_CPOX=2)
    vaccinated_girls_with_chickenpox = len(df[(df['SEX'] == 2) & (df['P_NUMVRC'] >= 1) & (df['HAD_CPOX'] == 1)])
    vaccinated_girls_without_chickenpox = len(df[(df['SEX'] == 2) & (df['P_NUMVRC'] >= 1) & (df['HAD_CPOX'] == 2)])

    # Calcular el ratio por género
    ratio_by_sex = {
        "male": vaccinated_boys_with_chickenpox / vaccinated_boys_without_chickenpox,
        "female": vaccinated_girls_with_chickenpox / vaccinated_girls_without_chickenpox
    }

    # Imprimir el diccionario de ratios
    print("Diccionario de Ratios:", ratio_by_sex)
    
    # Interpretación de los resultados
    interpretacion = "El porcentaje de niños que contrajeron varicela después de darse la vacuna es del {:.2%} en los hombres y del {:.2%} en las mujeres."

    return interpretacion.format(ratio_by_sex["male"], ratio_by_sex["female"])

# Llamar a la función para obtener la interpretación
interpretacion = chickenpox_by_sex()
print(interpretacion) # Interpretación del diccionario

Diccionario de Ratios: {'male': 0.009675583380762664, 'female': 0.0077918259335489565}
El porcentaje de niños que contrajeron varicela después de darse la vacuna es del 0.97% en los hombres y del 0.78% en las mujeres.
